In [9]:
import datetime
import json
import os

import requests
from bs4 import BeautifulSoup

'''
getConn: async with aiohttp.ClientSession(headers=self.headers)as session: 异步连接， 使用etree.HTML解析
GetZhihu: 解析soup，item.xpath 收集title, content
'''


class CrawData:
    def __init__(self):
        self.cookie = {
            "Zhihu": "_zap=76b4e1b6-cb41-474f-834c-02d66f618f59; d_c0=AODXtXv8fRWPTu9xBoST968twuJBcu0m9ig=|1662012047; YD00517437729195%3AWM_TID=wFNtumLPnL5FQQQBUQOBHP5%2FiBp5zUuH; _9755xjdesxxd_=32; gdxidpyhxdE=6%2Bda76P0N0buBJRKjOcYfpukmBT3a2BxiywALo2HZlrAdNwoGwkW7ZL8ZXda7IBMrfwLQvaX6ybQzEnsQ20v6lEafDT%2BHdnQEEXHNi3QfSgJJiHT%5C57%2FJ8Pga6ab%2BkMjYRiwDLtuNdOc%2FD2ZU7HI567zgXMGuvckJ21yn2M1xSIm7%2BvZ%3A1662106874351; YD00517437729195%3AWM_NI=BJurDgs5HdhkuGhxA%2BFP2hKQfQStlni5Xjp1A7ma0aao526nKQ7ZLhAbmlM7pnJDGPaua2j7cIqRrD5QX7UjvRWCzpG43Erbn2n66kEz6Jz%2F0mmb6xqoxmRxCJMVO9HFM00%3D; YD00517437729195%3AWM_NIKE=9ca17ae2e6ffcda170e2e6ee96b63a9baea4d0ed5c87bc8bb2c15f879f9e87c44da7eafda9ae4eb6ec86a9db2af0fea7c3b92a8296bb93c76db1eb8885eb61b799a899c4218690fe89ca5ba68f9d95ef66abeb988bf33b83e9a382b36ba3b1fbd3b566898ff9bbcf6483bea38dd03db4b286a5c65ca3abafd3e93da197bfb3f150f78f98d8d862b68e99a4f97c85ecfda6f16aaeeea2d7b56187a6faabf925f4b3a383b574b3b1b88ed174819683b7ce338c9a83d1b337e2a3; q_c1=9fde163d184f43c89b2a39878cb1191d|1662109910000|1662109910000; z_c0=2|1:0|10:1666873564|4:z_c0|80:MS4xVk4wbUJRQUFBQUFtQUFBQVlBSlZUWmhjUDJTMkg1cC1iY1drVzdvZ2xfN0pwdXpDLVV5TDJRPT0=|ba6373c3d19d92abc6a7d76e92e8c5786f16919d343b4f1447342b6ab6bd4aa2; _xsrf=2258b675-cd3b-4382-8635-3ac643942848; Hm_lvt_98beee57fd2ef70ccdd5ca52b9740c49=1666934408,1667016917,1667133485,1667198106; tst=h; NOT_UNREGISTER_WAITING=1; SESSIONID=7gjoDb6eWN6CFH57DlXdOd6ooIfAbLPYRIoUvO1isgP; JOID=V1wVAU7JIVGF7_XcdM7Qgx5qDqBk_0I57auW7hOebmPErr3oRzBQC-Hu8Npx-DMVFTGT-4XBQB7JZMDYRGbua2Y=; osd=W18cA0PFIliH4vnffczdjx1jDK1o_Es74KeV5xGTYmDNrLDkRDlSBu3t-dh89DAcFzyf-IzDTRLKbcLVSGXnaWs=; Hm_lpvt_98beee57fd2ef70ccdd5ca52b9740c49=1667201283; KLBRSID=3d7feb8a094c905a519e532f6843365f|1667201283|1667198101",
            "Weibo": "SINAGLOBAL=9561045312014.672.1664548752421; UOR=,,www.google.com; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9WFKGQYbnVAGgRIAI_cEZzbr5JpX5KMhUgL.Fo-Reo2XSo.X1hn2dJLoI7LCUgpE-NWDd.Bt; ULV=1668326704649:4:2:1:9211475239532.592.1668326704640:1668219191555; webim_unReadCount=%7B%22time%22%3A1668331056079%2C%22dm_pub_total%22%3A60%2C%22chat_group_client%22%3A157%2C%22chat_group_notice%22%3A0%2C%22allcountNum%22%3A275%2C%22msgbox%22%3A0%7D; PC_TOKEN=04d8bf9ca0; ALF=1699938496; SSOLoginState=1668402498; SCF=AgRxuG63DyJVy6c3vWUVplWvLg66D1s5oNRbGM3_tqCpr2R_MgRKkEi3CxJEVWof3aaSXPapLRkROvI3R3QJ2RI.; SUB=_2A25Odb0VDeRhGeNG6VMV9ifIwzSIHXVtAqndrDV8PUNbmtAKLVHDkW9NS2Aj9iZEMb0c9uEL2WPLi1w9Wq3-5Mx5; XSRF-TOKEN=3sTlo_roBUV1n2ql8KTPc0-M; WBPSESS=3D-OIkJPmA_YQyxuIGJtpblMKWzvV744TalsPfUZX7wFEf1_ai2q65MimN9tPI975XteLqDKkBOMwBFn-SQUKZ-PsQbiH46gyny2GBwFfR_FdUY3g-fmyj9SE1ck-67Q1J7Pqk5DOOCilCrexylCiA=="
        }
        self.urls = {
            'Zhihu': r'https://www.zhihu.com/hot',
            'Weibo': r'https://s.weibo.com/top/summary',
        }
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36',
        }
        self.data = {}

    def getDate(self):
        return str(datetime.date.today())

    def getConn(self, name):
        url = self.urls[name]
        # connect asynchronize
        text = requests.get(url, headers=self.headers).text
        return BeautifulSoup(text)

    def getZhihu(self):
        self.headers['cookie'] = self.cookie['Zhihu']
        soup = self.getConn("Zhihu")
        hotList = soup.find(attrs={"class": "HotList-list"})
        sections = hotList.findAll("section", limit=50)
        titles = []
        date2title = {}
        for section in sections:
            title = section.find(attrs={"class": "HotItem-title"}).text
            titles.append(title)
        date2title[self.getDate()] = titles
        self.data["Zhihu"] = date2title

    def getWeibo(self):
        self.headers["cookie"] = self.cookie["Weibo"]
        soup = self.getConn("Weibo")
        rows = soup.find(class_="data").findAll(class_="td-02", limit=50)
        titles = []
        date2title = {}
        for row in rows:
            title = row.find("a").text
            titles.append(title)
        date2title[self.getDate()] = titles
        self.data["Weibo"] = date2title

craw_data = CrawData()

In [14]:
zhihu = craw_data.getZhihu()
craw_data.data['Zhihu']

{'2022-11-14': ['中国高血压诊断标准下调，患者数量翻倍或近 5 亿，这数据说明了什么？我国高血压问题这么严重吗？',
  '如何看待 C 罗近期接受采访时「炮轰」曼联，表示自己感到被背叛？',
  '马斯克突然开除 80% 推特合同工，共计 4400 人，这将带来哪些影响？',
  '今年双 11，各大平台、主播战报不再强调 GMV，而是强调年轻用户数和时长，透露了哪些信号？',
  '儿子说「我只想做个普通人」，该怎么教育孩子？',
  '为什么有的计算机本科生就能发表顶会，但是许多博士生还是一篇都没有？',
  '11 月 13 日北京新增 237 例本土确诊、 170 例本土无症状感染者，目前疫情情况如何？',
  '防疫发布会被质疑打广告，乌鲁木齐市商务局回应「有企业断章取义」，该企业可能面临哪些处罚？',
  '退休教师三亚全款买 22 套房 10 年未拿到，仲裁交房开发商「不认可」，遇此情况该如何维权？',
  '11 月 13 日重庆新增本土感染病例「150+2147」例 ，目前当地疫情情况如何？',
  '潮州特斯拉失控致 2 死 3 伤，特斯拉回应「踩不下刹车」，对于此事件你有哪些看法？',
  '如何看待 Nature communication 和 Science advances 降为二区？',
  '成都一儿童医院现付费「视频座椅」引关注，9 元可看一小时，「视频座椅」的出现，对就医体验带来哪些改变？',
  '医护人员住宾馆遭暴力驱赶，大同警方回应称相关人员已被采取强制措施，具体情况如何？如何看待此事件？',
  '网友称得物调用手机权限删投诉视频，得物回应「没有权限和能力，正与手机厂商确认」，哪些信息值得关注？',
  '如何看待上海 F4（米哈游，莉莉丝，鹰角，叠纸）?',
  '湖南一社区书记同意转卖捐赠物资被免职，起到哪些警示作用？',
  '什么原因让年轻人特别在乎钱？',
  '土耳其一地突发爆炸已致 6 死 81 伤，土耳其副总统称「这是恐怖袭击」，目前当地情况如何？',
  '11 月 14 日是世界糖尿病日，为什么中国成了糖尿病王国？我们又该如何做好糖尿病的早期预防？',
  '为什么现在有不少人怀念 Win10，抵制 Win11。和几年前怀念 Win7，抵制 Win8/10 很相似？',
  '研究称「

In [10]:
craw_data.getWeibo()
craw_data.data['Weibo']

{'2022-11-14': ['让探索太空的脚步迈得更稳更远',
  '石家庄疫情防控',
  '小猫叼巨型蟑螂回家躺床抱着玩',
  '生态文明与美丽中国实践边会',
  '鹿晗官宣音乐会嘉宾',
  '妖猫传里有陈飞宇',
  '偷偷藏不住',
  '马斯克突然开除80%推特合同工',
  '父亲婚礼致辞紧张到全身发抖',
  '广州新增本土感染者4065例',
  '乒乓球世界冠军朱雨玲病愈归来',
  '女子已生4女儿又怀孕被要求离婚',
  '白敬亭尹峥原型是雍正',
  '爱的二八定律开播追剧日历',
  '前YG社长杨贤硕有期徒刑3年',
  '这锅包肉能全国巡炒吗',
  '95后宝妈带1岁大宝宝上班',
  '白血病男子术后6年还清70万治病钱',
  '女子老房拆迁挖出千年古墓',
  '篮网vs湖人',
  '8岁男童右眼感染20多条螨虫',
  '金韩彬',
  '倪妮海藻卷发',
  '偷偷藏不住全员杀青照',
  '睡眠不足真的会发胖',
  '首批新员工抵达郑州富士康',
  '北京新增48例社会面',
  '这老师是怎么忍住不笑的',
  '重庆新增本土确诊150例无症状2147例',
  '橄榄球赛韩国站播错中国国歌',
  'C罗炮轰曼联',
  '陈采尼金鸡奖点评',
  '大学寝室被李峋包围',
  '杨幂许凯新剧反差感恋爱',
  '赵丽颖欧豪风吹半夏开始预约',
  '陈玘',
  '黄明昊给王鹤棣沈月拍合影',
  '这下不想养猫了',
  '用不明液体泼洒路人2男子被拘',
  '男童受气后牵狗徒步17公里找奶奶',
  '苹果开发搜索引擎',
  '时代少年团新歌与奥特曼适配度',
  '医生飞针采血被赞蚊子投胎',
  '恩比德59分',
  '北京疫情防控',
  '老人地铁上不顾劝阻用刀削苹果',
  '北京朝阳区部分核酸检测点临时关闭',
  '广东新增本土感染者4160例',
  '每天都会来串门的小奶狗',
  '男子吃自助餐发现餐桌立牌请勿摄像']}

In [11]:
import json
import os
with open(os.getcwd() + "/wc_cn/weiboHot.txt", "a+", encoding="utf-8") as file:
    file.write(json.dumps(craw_data.data['Weibo'], indent=4, ensure_ascii=False))

In [15]:
import json
import os
with open(os.getcwd() + "/wc_cn/zhihuHot.txt", "a+", encoding="utf-8") as file:
    file.write(",\n"+json.dumps(craw_data.data['Zhihu'], indent=4, ensure_ascii=False))